In [127]:
import pandas as pd
import glob
import re

ticks = [
    pd.read_csv(f, index_col='Date').Close.fillna(method='ffill').rename(f)
    for f in glob.glob('data/*.yahoo.*')
]

ticks = [t.rename(re.match('.+/([^.]+).+', t.name).group(1)) for t in ticks]
ticks = [t[~t.index.duplicated()] for t in ticks]

ticks_df = pd.concat(ticks, join='inner', axis=1)
change_df = ticks_df.pct_change().dropna()
change_df.corr().style.background_gradient(cmap='Blues')

,CL=F,NVDA,ZEC-USD,GC=F,BTC-USD,^GSPC,^IXIC,RUB=X
CL=F,1.000000,0.050126,0.073205,0.097908,-0.004732,0.193626,0.145311,0.027445
NVDA,0.050126,1.000000,-0.043857,-0.136660,0.034880,0.533916,0.618540,0.052234
ZEC-USD,0.073205,-0.043857,1.000000,0.094506,0.243015,0.003308,0.009106,-0.097248
GC=F,0.097908,-0.136660,0.094506,1.000000,0.030614,-0.064365,-0.042447,-0.104005
BTC-USD,-0.004732,0.034880,0.243015,0.030614,1.000000,0.028405,0.027146,0.016147
^GSPC,0.193626,0.533916,0.003308,-0.064365,0.028405,1.000000,0.946041,-0.043644
^IXIC,0.145311,0.618540,0.009106,-0.042447,0.027146,0.946041,1.000000,-0.027194
RUB=X,0.027445,0.052234,-0.097248,-0.104005,0.016147,-0.043644,-0.027194,1.000000
